# Reccomendation System Testing

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Fashion items
data = [
    {"id": 1, "name": "Black T-Shirt", "tags": "casual cotton minimal"},
    {"id": 2, "name": "White Hoodie", "tags": "winter streetwear cozy"},
    {"id": 3, "name": "Denim Jacket", "tags": "outerwear classic blue"},
    {"id": 4, "name": "Blue Jeans", "tags": "denim everyday casual"},
]

df = pd.DataFrame(data)

# TF–IDF vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["tags"])

# Query
query = ["casual denim"]
query_vec = vectorizer.transform(query)

# Similarity scores
sims = cosine_similarity(query_vec, tfidf_matrix)[0]
df["similarity"] = sims

# Sort results
results = df.sort_values("similarity", ascending=False)

# Clean formatted print
print(f"\n--- Recommended Items to pair with {query} ---\n")
for _, row in results.iterrows():
    print(f"{row['id']} | {row['name']} | {row['tags']} | Similarity: {row['similarity']:.3f}")



--- Recommended Items to pair with ['casual denim'] ---

4 | Blue Jeans | denim everyday casual | Similarity: 0.786
1 | Black T-Shirt | casual cotton minimal | Similarity: 0.301
2 | White Hoodie | winter streetwear cozy | Similarity: 0.000
3 | Denim Jacket | outerwear classic blue | Similarity: 0.000


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

df = pd.DataFrame([
    {"id": 1, "name": "Black T-Shirt", "color_darkness": 0.9, "formality": 0.2, "price": 15},
    {"id": 2, "name": "White Hoodie", "color_darkness": 0.1, "formality": 0.3, "price": 45},
    {"id": 3, "name": "Denim Jacket", "color_darkness": 0.5, "formality": 0.5, "price": 60},
    {"id": 4, "name": "Blue Jeans", "color_darkness": 0.6, "formality": 0.4, "price": 40},
])

# Normalize numeric features
scaler = StandardScaler()
features = scaler.fit_transform(df[["color_darkness", "formality", "price"]])

# Query: prefer medium-dark, semi-casual, affordable
query = [[0.5, 0.4, 40]]
sims = cosine_similarity(query, features)[0]
df["similarity"] = sims

# Sort and print
results = df.sort_values("similarity", ascending=False)
print(f"Embeddings:{query}\n")
print("\n--- Recommended Items ---\n")
for _, row in results.iterrows():
    print(f"{row['id']} | {row['name']} | Price: ${row['price']} | Similarity: {row['similarity']:.3f}")


Embeddings:[[0.5, 0.4, 40]]


--- Recommended Items ---

3 | Denim Jacket | Price: $60 | Similarity: 0.683
2 | White Hoodie | Price: $45 | Similarity: 0.181
4 | Blue Jeans | Price: $40 | Similarity: 0.015
1 | Black T-Shirt | Price: $15 | Similarity: -0.634


# Netflix recommendation style - Matrix Factorization

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# Sample user-item ratings
ratings_data = [
    {"user_id": 1, "item_id": 1, "rating": 5},
    {"user_id": 1, "item_id": 2, "rating": 3},
    {"user_id": 1, "item_id": 3, "rating": 4},
    {"user_id": 2, "item_id": 1, "rating": 4},
    {"user_id": 2, "item_id": 3, "rating": 5},
    {"user_id": 2, "item_id": 4, "rating": 2},
    {"user_id": 3, "item_id" : 2, "rating": 4},
    {"user_id": 3, "item_id": 4, "rating": 5},
]
ratings_df = pd.DataFrame(ratings_data)
items_df = pd.DataFrame([
    {"item_id": 1, "name": "Black T-Shirt"},
    {"item_id": 2, "name": "White Hoodie"},
    {"item_id": 3, "name": "Denim Jacket"},
    {"item_id": 4, "name": "Blue Jeans"},
])
# Create user-item matrix
user_item_matrix = ratings_df.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
sparse_matrix = csr_matrix(user_item_matrix.values)
# Apply NMF
nmf = NMF(n_components=2, init='random', random_state=42)
user_features = nmf.fit_transform(sparse_matrix)
item_features = nmf.components_.T
# Normalize features
scaler = MinMaxScaler()
user_features = scaler.fit_transform(user_features)
item_features = scaler.fit_transform(item_features)
# Recommend for user_id=1
user_id = 1
user_idx = user_id - 1
user_vec = user_features[user_idx].reshape(1, -1)
sims = cosine_similarity(user_vec, item_features)[0]
items_df["similarity"] = sims
results = items_df.sort_values("similarity", ascending=False)
# Print results
print(f"\nUser {user_id} Recommendations:\n")
print("\n--- Recommended Items ---\n")
for _, row in results.iterrows():
    print(f"{row['item_id']} | {row['name']} | Similarity: {row['similarity']:.3f}")
    



User 1 Recommendations:


--- Recommended Items ---

1 | Black T-Shirt | Similarity: 0.998
3 | Denim Jacket | Similarity: 0.998
2 | White Hoodie | Similarity: 0.254
4 | Blue Jeans | Similarity: 0.060
